In [18]:
"""Mini framework that implements auto differentiation of math expression defined with custom object Node.
Also implements fully connected neural network model, optimizer, loss function.
"""

'Mini framework that implements auto differentiation of math expression defined with custom object Node.\nAlso implements fully connected neural network model, optimizer, loss function.\n'

In [21]:
import numpy as np
import random

In [35]:
class Node:
    """Composed class for defining math expressions and automatic differentiation.

    Args:
        val (int | float): Value to store in Node object; will be used in mathematical operations.
        label (str): String label to identify instance of Node.
        grad (float): Gradient of current Node, default is 0.
    """
    def __init__(
        self, 
        val: int | float, 
        label: str, 
        _components: tuple['Node'] = (),
        ):
        self.val = val
        self.label = label
        self.grad = 0
        self._components = _components
        self._backward = lambda: None
        
    def __repr__(self):
        return f"Node(val={self.val})"
    
    def __add__(self, other):
        """Defines addition of two Nodes and sets up gradient computation.

        Args:
            other (Node or float or int): Node or numeric value to add.

        Returns:
            Node: Resultant Node with updated computation graph.
        """
        other = other if isinstance(other, Node) else Node(val=other, label=f'_{self.label}_')
        new_node = Node(val=self.val + other.val, label=f'({self.label} + {other.label})', _components=(self, other))
        def _backward():
            self.grad += new_node.grad
            other.grad += new_node.grad
        new_node._backward = _backward
        return new_node
    
    def __sub__(self, other):
        """Defines subtraction of two Nodes and sets up gradient computation.

        Args:
            other (Node or float or int): Node or numeric value to subtract.

        Returns:
            Node: Resultant Node with updated computation graph.
        """
        other = other if isinstance(other, Node) else Node(val=other, label=f'_{self.label}_')
        new_node = Node(val=self.val - other.val, label=f'({self.label} - {other.label})', _components=(self, other))
        def _backward():
            self.grad += new_node.grad
            other.grad -= new_node.grad
        new_node._backward = _backward
        return new_node
    
    def __mul__(self, other):
        """Defines multiplication of two Nodes and sets up gradient computation.

        Args:
            other (Node or float or int): Node or numeric value to multiply.

        Returns:
            Node: Resultant Node with updated computation graph.
        """
        other = other if isinstance(other, Node) else Node(val=other, label=f'_{self.label}_')
        new_node = Node(val=self.val * other.val, label=f'({self.label} * {other.label})', _components=(self, other))
        def _backward():
            self.grad += new_node.grad * other.val
            other.grad += new_node.grad * self.val
        new_node._backward = _backward
        return new_node
    
    def __pow__(self, other):
        """Raises the Node to a given power and sets up gradient computation.

        Args:
            other (float): Power to which the node is raised.

        Returns:
            Node: Resultant Node with updated computation graph.
        """
        new_node = Node(val=self.val ** other, label=f'({self.label}^{other})', _components=(self,))
        def _backward():
            self.grad += new_node.grad * (other * self.val ** (other - 1))
        new_node._backward = _backward
        return new_node

    def __truediv__(self, other):
        return self * other ** -1
    
    def __neg__(self):
        return self * -1
    
    def __radd__(self, other):
        return self + other
    
    def __rmul__(self, other):
        return self * other
    
    def __rtruediv__(self, other):
        return self * other**-1
    
    def _topological_sort(self):
        """Returns list of all Nodes forming the current Node in topological order."""
        topologically_sorted_nodes = []
        traversed_nodes = set()
        
        def traverse(node):
            """Implements DFS for topological sorting."""
            if not node in traversed_nodes:
                traversed_nodes.add(node)
                for component in node._components:
                    traverse(component)
                topologically_sorted_nodes.append(node)
        traverse(self)
        return topologically_sorted_nodes
    
    def sigmoid(self):
        """Applies sigmoid activation function to the current Node.

        Returns:
            Node: Node with sigmoid transformation applied.
        """
        new_val = 1 / (1 + np.exp(-self.val))
        new_node = Node(val=new_val, label=f'sigmoid({self.label})', _components=(self,))
        def _backward():
            self.grad += new_node.grad * (new_val * (1 - new_val))
        new_node._backward = _backward
        return new_node
    
    def tanh(self):
        """Applies hyperbolic tangent activation function to the current Node.

        Returns:
            Node: Node with tanh transformation applied.
        """
        new_val = (np.exp(self.val) - np.exp(-self.val)) / (np.exp(self.val) + np.exp(-self.val))
        new_node = Node(val=new_val, label=f'tanh({self.label})', _components=(self, ))
        def _backward():
            self.grad += new_node.grad * (1 - new_val**2)
        new_node._backward = _backward
        return new_node

    def backward(self):
        """Performs backpropagation to compute gradients."""
        self.grad = 1.0
        topologicaly_sorted_nodes = self._topological_sort()
        for node in topologicaly_sorted_nodes[::-1]:
            node._backward()
    
    @property
    def info(self):
        """Returns string with Node details."""
        return f"Function: '{self.label}' | Value: {self.val} | Grad: {self.grad}"
    
    @property
    def computational_graph(self):
        """Prints the computational graph in reverse topological order."""
        topologicaly_sorted_nodes = self._topological_sort()
        for node in topologicaly_sorted_nodes[::-1]:
            print(node.info)
    
    
# Example
a = Node(val=7.0, label='a')
b = Node(val=-3.0, label='b')
c = Node(val=5.0, label='c')
s = a*b + c
o = s.tanh()
o.backward()
o.computational_graph

Function: 'tanh(((a * b) + c))' | Value: -0.9999999999999748 | Grad: 1.0
Function: '((a * b) + c)' | Value: -16.0 | Grad: 5.040412531798211e-14
Function: 'c' | Value: 5.0 | Grad: 5.040412531798211e-14
Function: '(a * b)' | Value: -21.0 | Grad: 5.040412531798211e-14
Function: 'b' | Value: -3.0 | Grad: 3.5282887722587475e-13
Function: 'a' | Value: 7.0 | Grad: -1.5121237595394632e-13


In [38]:
class Neuron:
    """Defines a single Neuron for a Neural Network."""
    
    _activations = {
        'sigmoid': lambda node: node.sigmoid(),
        'tanh': lambda node: node.tanh()
        }
    
    def __init__(self, n_inputs: int, activation: str = 'sigmoid'):
        """Initializes a Neuron.

        Args:
            n_inputs: Number of input connections.
            activation: Activation function to use ('sigmoid' or 'tanh').
        """
        self.w = [Node(random.uniform(-1, 1), label=f'neuron_w{i}') for i in range(n_inputs)]
        self.b = Node(0, label='b')
        self.activation = activation
        
    def __call__(self, x):
        """Computes output as activation(dot(w, x) + b).

        Args:
            x: List of input Node values.

        Returns:
            Activated output Node.
        """
        activation = self._activations[self.activation]
        return activation(sum([wi*xi for wi, xi in zip(self.w, x)]) + self.b)

    def __repr__(self):
        return f'Neuron(n_inputs={len(self.w)})'
    
    @classmethod
    def _add_activation(cls, activation, activation_name):
        """Adds a new activation function to Neuron.

        Args:
            activation: Callable to use as activation function.
            activation_name: Name to register the activation under.
        """
        cls._activations[activation_name] = activation

    def parameters(self):
        """Returns a list of learnable parameters (weights and bias).

        Returns:
            List of Node instances (weights and bias).
        """
        return self.w + [self.b]
    
class Layer:
    """Defines a Layer of a Neural Network.

    Args:
        n_inputs: Dimensionality of input.
        n_outputs: Dimensionality of output.
        activation: Activation function used in every node of Layer.
    """
    def __init__(
        self, 
        n_inputs: int, 
        n_outputs: int,
        activation: str = 'sigmoid'
        ):
        self.n_inputs = n_inputs
        self.n_outputs = n_outputs
        self.activation = activation
        self.neurons = [Neuron(n_inputs, activation) for _ in range(n_outputs)]
        
    def __repr__(self):
        return f'Layer(n_inputs={self.n_inputs}, n_outputs={self.n_outputs}, activation={self.activation})'
    
    def __call__(self, x):
        """Calculates the output of the Neurons in the Layer.

        Args:
            x: Input values for the Layer.

        Returns:
            Output from the Layer; single value if only one neuron.
        """
        out = [neuron(x) for neuron in self.neurons]
        return out[0] if len(out) == 1 else out
    
    def parameters(self):
        """Returns a list of all learnable parameters in the Layer.

        Returns:
            List of Node instances (weights and biases).
        """
        return [param for neuron in self.neurons for param in neuron.parameters()]
    
    
class MLP:
    """Defines a fully connected Multi-Layer Perceptron.

    Args:
        n_inputs: Dimensionality of input.
        hidden_layers: List of hidden layer sizes.
        n_outputs: Dimensionality of output layer.
        activations: Activation function(s) used in each layer.
    """
    def __init__(self, n_inputs, hidden_layers, n_outputs, activations = 'sigmoid'):
        self.n_inputs = n_inputs
        self.hidden_layers = hidden_layers
        self.n_outputs = n_outputs
        total_dims = [self.n_inputs] + self.hidden_layers + [self.n_outputs]
        self.activations = activations if not activations == 'sigmoid' else ['sigmoid' for _ in range(len(total_dims) - 1)]
        self.layers = [
            Layer(n_inputs=total_dims[i - 1], n_outputs=total_dims[i], activation=self.activations[i - 1]) 
            for i in range(1, len(total_dims))
            ]
        
    def __repr__(self):
        return (f'MLP(\n'
                f'    n_inputs={self.n_inputs},\n'
                f'    hidden_layers={self.hidden_layers},\n'
                f'    n_outputs={self.n_outputs},\n'
                f'    activations={self.activations}\n'
                f')')
    
    def __call__(self, x):
        """Performs a forward pass through the network.

        Args:
            x: Input values.

        Returns:
            Output after forward pass.
        """
        for layer in self.layers:
            x = layer(x)
        return x
    
    def parameters(self):
        """Returns a list of all learnable parameters in the network.

        Returns:
            List of Node instances (weights and biases).
        """
        return [param for layer in self.layers for param in layer.parameters()]

# Example
model = MLP(
    n_inputs=2, 
    hidden_layers=[2], 
    n_outputs=1, 
    activations=['sigmoid', 'sigmoid']
    )

x = [1.0, 1.0]
model(x)

Node(val=0.7160178812275462)

In [39]:
class Optimizer:
    """Performs a gradient descent step on provided parameters.

    Attributes:
        parameters: List of parameters to update.
        learning_rate: Step size for gradient descent.
    """
    def __init__(self, parameters, learning_rate = 0.001):
        """Initializes the Optimizer.

        Args:
            parameters: Callable that returns a list of parameters (e.g., model.parameters).
            learning_rate: Learning rate for gradient update.
        """
        self.parameters = parameters
        self.learning_rate = 0.001
        
    def __repr__(self):
        return f'Optimizer(learning_rate={self.learning_rate})'
        
    def zero_grad(self):
        """Resets gradient of all parameters to zero."""
        for parameter in self.parameters(): parameter.grad = 0
        
    def step(self):
        """Performs one step of gradient descent update."""
        for parameter in self.parameters(): parameter.val -= self.learning_rate * parameter.grad
            
optimizer = Optimizer(model.parameters)
optimizer

Optimizer(learning_rate=0.001)

In [40]:
class MSE:
    """Mean Squared Error (MSE) loss function."""
    
    def __call__(self, y_true, y_pred):
        """Computes the mean squared error between predictions and targets.

        Args:
            y_true: Iterable of true target values.
            y_pred: Iterable of predicted values.

        Returns:
            The mean squared error as a float.
        """
        return sum(((y_pred - y_true)**2 for y_true, y_pred in zip(y_true, y_pred))) / len(y_true)
        
    def __repr__(self):
        return f'MSE()'
    
loss = MSE()
loss

MSE()

In [43]:
# Example

# Data to work with
xs = [
    [2.0, 3.0, -1.0],
    [3.0, -1.0, 0.5]
      ]
ys = [1.0, -1.0]


# Loss function
mse = MSE()

# Optimization
loss = mse(ys, [model(x) for x in xs])
for _ in range(5):
  print(f'Loss before: {loss}')
  optimizer.zero_grad()
  loss.backward()
  optimizer.step()
  loss = mse(ys, [model(x) for x in xs])
  print(f'Loss after: {loss}\n')


Loss before: Node(val=1.4228374421423402)
Loss after: Node(val=1.4226470926950001)

Loss before: Node(val=1.4226470926950001)
Loss after: Node(val=1.4224567403775608)

Loss before: Node(val=1.4224567403775608)
Loss after: Node(val=1.4222663852823236)

Loss before: Node(val=1.4222663852823236)
Loss after: Node(val=1.4220760275016133)

Loss before: Node(val=1.4220760275016133)
Loss after: Node(val=1.421885667127775)



In [45]:
loss.computational_graph

Function: '(((((sigmoid(((((neuron_w0 * sigmoid(((((neuron_w0 * _neuron_w0_) + _(neuron_w0 * _neuron_w0_)_) + (neuron_w1 * _neuron_w1_)) + b))) + _(neuron_w0 * sigmoid(((((neuron_w0 * _neuron_w0_) + _(neuron_w0 * _neuron_w0_)_) + (neuron_w1 * _neuron_w1_)) + b)))_) + (neuron_w1 * sigmoid(((((neuron_w0 * _neuron_w0_) + _(neuron_w0 * _neuron_w0_)_) + (neuron_w1 * _neuron_w1_)) + b)))) + b)) - _sigmoid(((((neuron_w0 * sigmoid(((((neuron_w0 * _neuron_w0_) + _(neuron_w0 * _neuron_w0_)_) + (neuron_w1 * _neuron_w1_)) + b))) + _(neuron_w0 * sigmoid(((((neuron_w0 * _neuron_w0_) + _(neuron_w0 * _neuron_w0_)_) + (neuron_w1 * _neuron_w1_)) + b)))_) + (neuron_w1 * sigmoid(((((neuron_w0 * _neuron_w0_) + _(neuron_w0 * _neuron_w0_)_) + (neuron_w1 * _neuron_w1_)) + b)))) + b))_)^2) + _((sigmoid(((((neuron_w0 * sigmoid(((((neuron_w0 * _neuron_w0_) + _(neuron_w0 * _neuron_w0_)_) + (neuron_w1 * _neuron_w1_)) + b))) + _(neuron_w0 * sigmoid(((((neuron_w0 * _neuron_w0_) + _(neuron_w0 * _neuron_w0_)_) + (neur